In [1]:
import numpy as np
import pandas as pd
import matplotlib
from collections import defaultdict
from matplotlib import pyplot as plt
from urllib import urlencode

import urllib2
import time
import json
import operator
import urllib2 

In [2]:
file = open('datasets/token', 'r')
accessToken = file.read()

# Достаем полицейских из закрытых групп

In [4]:
groups = ["109490789", 
          "2100831", 
          "11912933", 
          "1167943", 
          "15335079", 
          "12813079", 
          "68478644",
          "16vzvoduvo",
          "59565067",
          "61511779",
          "1182973",
          "1119648",
          "21754234",
          "14294712",
          "7591998",
          "8702762",
          "23386497",
          "1566321",
          "55523638"]

users = []

for group in groups:
    params = {"group_id": group, 
              "count": 500, 
              "fields": "sex,education,universities,schools",
              "access_token": accessToken}
    params = urlencode(params)

    url = "https://api.vk.com/method/groups.getMembers?" + params
    data = json.load(urllib2.urlopen(url))
    data_users = data['response']['users']
    
    print("extracted %d users from %s" % (len(data_users), group))
    users.extend(data_users)
    time.sleep(0.5)
    
print("sum: %d" % len(users))

f = open('datasets/police/users.json','w')
f.write(json.dumps(users))
f.close()

URLError: <urlopen error EOF occurred in violation of protocol (_ssl.c:590)>

# Достаем случайных людей из Петербурга

In [ ]:
people = []

while len(people) < 2500:
    random_ids = []
    for i in range(0, 100):
        random_ids.append(random.randint(1, 90000000))

    user_ids = ",".join(str(x) for x in random_ids)
    params = {"user_ids": user_ids, 
              "fields": "sex,education,universities,schools,city", 
              "access_token": accessToken}
    params = urlencode(params)

    url = "https://api.vk.com/method/users.get?" + params
    data = json.load(urllib2.urlopen(url))
   
    fetched_people = data.get('response', [])
    fetched_people = filter(lambda x: x.get('deactivated', '') == '', fetched_people)
    fetched_people = filter(lambda x: x.get('city', 0) == 2, fetched_people)
    
    print("extracted %d users" % len(fetched_people))
    people.extend(fetched_people)
    time.sleep(0.25)
    
f = open('datasets/people/users.json','w')
f.write(json.dumps(people))
f.close()

# Достаем группы

In [5]:
def get_groups(users_file, save_to):
    with open(users_file) as data_file:    
        users = json.load(data_file)

    groups = {}

    print("processing %d users" % len(users))
    for i in range(0, len(users)):
        params = {"user_id": users[i]['uid'], 
                  "extended": 0, 
                  "access_token": accessToken}
        params = urlencode(params)

        url = "https://api.vk.com/method/groups.get?" + params
        data = json.load(urllib2.urlopen(url))

        if data.get('response', ''):
            user_groups = data['response']
            for user_group in user_groups:
                groups[user_group] = groups.get(user_group, 0) + 1

            print "%d" % (i + 1),
        time.sleep(0.5)

    f = open(save_to,'w')
    f.write(json.dumps(groups))
    f.close()

In [ ]:
get_groups('datasets/police/users.json', 'datasets/police/groups_ids.json')

In [6]:
get_groups('datasets/people/users.json', 'datasets/people/groups_ids.json')

processing 2503 users
1 5 7 9 10 13 14 18 20 21 22 23 24 25 28 33 37 38 41 42 43 44 46 48 49 50 51 54 56 57 58 59 60 65 68 70 71 72 73 76 77 80 82 83 85 86 90 91 94 95 97 98 99 100 101 105 106 107 109 110 111 112 113 114 115 116 119 120 121 122 123 124 125 126 128 130 132 133 134 136 137 138 139 141 143 145 147 148 149 150 151 152 153 155 156 157 158 160 162 164 165 166 167 168 171 173 176 178 179 180 184 185 186 188 189 190 191 193 194 196 198 200 201 203 205 206 208 209 210 213 214 216 217 221 222 225 226 228 230 233 234 235 236 239 240 241 242 244 245 246 252 254 255 257 258 259 260 261 262 263 266 267 268 269 270 271 272 273 275 276 277 279 281 283 284 285 286 289 290 291 293 294 295 296 298 300 302 303 304 306 307 308 309 310 311 312 313 318 319 320 322 323 326 327 329 330 331 332 336 341 342 343 344 346 348 351 352 354 357 358 359 360 362 363 364 365 366 367 368 369 370 373 377 378 379 380 382 383 385 387 389 391 393 394 395 397 403 404 405 406 407 408 410 412 413 414 415 416 417

# Достаем названия групп

In [16]:
def get_group_names(group_ids_file, save_to):
    with open(group_ids_file) as data_file:    
        groups = json.load(data_file)

    sorted_groups = sorted(groups.items(), key=operator.itemgetter(1))
    sorted_groups.reverse()

    # ids of first 100 groups as string
    group_ids = ",".join(map(lambda x: x[0], sorted_groups[:100]))
    params = {"group_ids": group_ids}
    params = urlencode(params)
    url = "https://api.vk.com/method/groups.getById?" + params
    data = json.load(urllib2.urlopen(url))

    # function to "attach" people count to groups
    def add_people_count(group_info):
        group_info['people_count'] = groups.get(str(group_info['gid']), 0)
        return group_info

    if data.get('response', ''):
        groups_info = data['response']
        groups_info = map(add_people_count, groups_info)

        f = open(save_to,'w')
        f.write(json.dumps(groups_info))
        f.close()

In [ ]:
get_group_names('datasets/police/groups_ids.json', 'datasets/police/groups.json')

In [12]:
get_group_names('datasets/people/groups_ids.json', 'datasets/people/groups.json')

In [17]:
get_group_names('datasets/police/media_ids.json', 'datasets/police/media.json')

AttributeError: 'list' object has no attribute 'items'

In [14]:
get_group_names('datasets/people/media_ids.json', 'datasets/people/media.json')